## Terminology

- **Trial**: single occurrence with an outcome that is uncertain until we observe it.
For example, rolling a single die.
- **Outcome**: possible result of a trial; one particular state of the world. What Laplace calls a case.
For example: 4.
- **Sample Space**: The set of all possible outcomes for the trial.
For example, $\{1, 2, 3, 4, 5, 6\}$.
- **Event**: subset of the sample space, a set of outcomes that together have some property we are interested in.
For example, the event "odd die roll" is the set of outcomes $\{1,3,5\}$.
- **Probability**: As Laplace said, the probability of an event with respect to
  a sample space is the "number of favorable cases" (outcomes from the sample
  space that are in the event) divided by the "number of all the cases" in the
  sample space (assuming "nothing leads us to expect that any one of these cases
  should occur more than any other"), so $P \in [0,1]$.
  For example, the probability of an odd die roll is $3/6 = 1/2$.
- **Frequency**: non-negative number describing how often an outcome occurs. Can be a count like 5, or a ratio like 1/6.
- **Distribution**: mapping from outcome to frequency of that outcome. We will allow sample spaces to be distributions. 
- **Probability Distribution**: distribution whose frequencies sum to 1. 


## Introduction

Sampling table: 
  * with replacement: we can pick the same item again (rolling dice, coin flip)
  * without replacement: cannot choose again (pick a ball, team player...)


  &nbsp; | Order matters | Order doesn't matter
---------|----------|---------
 Replace | $n^k$ | $n+k-1 \choose k$
 Don't replace | $\frac{n!}{(n - k)!}$ | ${n \choose k} = \frac{n!}{(n - k)! \times k!}$

### die rolling implementation

In [263]:
from fractions import Fraction

def P(event, space):
    "The probability of an event, given a sample space."
    return Fraction(cases(favorable(event, space)),cases(space))

favorable = set.intersection # Outcomes that are in the event and in the sample space
cases     = len              # The number of cases is the length, or size, of a set

D     = {1, 2, 3, 4, 5, 6} # a sample space
even  = {   2,    4,    6} # an event
# NB: intersected with sample space to get favorable outcomes
odd   = {1, 3, 5, 7, 9, 11, 13}
prime = {2, 3, 5, 7, 11, 13}

In [264]:
P(even, D)

Fraction(1, 2)

In [265]:
P(odd, D)

Fraction(1, 2)

In [266]:
P((even | prime), D) # The probability of an even or prime die roll

Fraction(5, 6)

In [267]:
P((odd & prime), D) # The probability of an odd prime die roll

Fraction(1, 3)

### Card problems

In [268]:
import itertools
import random

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return list(set(map(' '.join, itertools.combinations(items, n))))

suits = u'♥♠♦♣'
ranks = u'AKQJT98765432'
deck  = [r + s for r in ranks for s in suits] # 52
# sample space of all 5-card combinations from deck
Hands = combos(deck, 5) # 2598960

In [269]:
random.sample(Hands, 7)


['A♦ K♣ 7♥ 5♠ 4♣',
 'T♠ 9♦ 8♦ 4♠ 3♣',
 'K♠ K♣ 8♣ 3♠ 3♣',
 'K♣ J♦ T♣ 9♠ 7♠',
 'A♣ Q♠ 7♦ 4♣ 3♦',
 'T♣ 8♠ 5♦ 3♥ 3♦',
 'T♦ 9♥ 6♥ 6♣ 3♠']

In [270]:
random.sample(deck, 7)

['K♣', '8♠', '3♠', 'K♠', '2♠', '4♠', '8♣']

In [271]:
# 5 cards of the same suit
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}
P(flush, Hands)

Fraction(33, 16660)

In [272]:
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}
P(four_kind, Hands) # e.g. 4 kings

Fraction(1, 4165)

## Urn Problems

> *An urn contains 6 blue, 9 red, and 8 white balls.  We select 6 balls at random. What is the probability of each of these  outcomes:*
> 
> - *All balls are red*.
> - *3 are blue, and 1 is red, and 2 are white, *.
> - *Exactly 4 balls are white*.

In [273]:
def balls(color, n):
    "A set of n numbered balls of the given color."
    return {color + str(i)
            for i in range(1, n + 1)}

urn = list(balls('B', 6) | balls('R', 9) | balls('W', 8))

U6 = combos(urn, 6) 
"""sample space (we select 6 random balls)"""

random.sample(U6, 1)

['B5 W5 B3 R1 B2 R4']

In [274]:
def select(color, n, space=U6):
    """
    The subset of the sample space with exactly `n` balls of given `color`.
    Example: `select('R', 6)` is the event of picking 6 red balls from the urn
    """
    return {s for s in space if s.count(color) == n}

In [275]:
# *All balls are red*.
P(select('R', 6), U6) 

Fraction(4, 4807)

In [276]:
# *3 are blue, and 1 is red, and 2 are white, *.
P(select('B', 3) & select('R', 1) & select('W', 2), U6)

Fraction(240, 4807)

In [277]:
# 4 balls (only) are white
P(select('W', 4), U6)


Fraction(350, 4807)

TODO: what about probability that 3 or more are white (not necessarily consecutive)? 

see
https://math.stackexchange.com/questions/1292902/the-probability-of-getting-at-least-5-balls-of-the-same-color-from-a-uniformly-d
which is basically the same exact problem.

we should first simplify the problem by grouping useless balls: i.e. combine blue
and red as "?".

The probability of any of the _+3 white balls_ event
(WWW???, ?W?WW?, ??WWW?, ...) is based on order matters without replacement




### Verifying urn calculations with arithmetic

Let's verify the first question of drawing exactly 6 red balls.

We need to first answer _how many ways can I choose 6 out of 9 red balls?_

$$
6 \choose 9
$$

If we care about the order in which they're drawn, the number of ways of choosing $k$ out of $n$ items is:

$$
\frac{n!}{(n - k)!}
$$

But if we don't care about the *order* of the six drawn balls, we can divide
that product by the number of permutations of 6 things, i.e. 6:

$$
\frac{n!}{(n - k)! \times k!} = {n \choose k}
$$





In [279]:
from math import factorial

def choose(n, k):
    "Number of ways to choose k items from a list of n items without taking order into account."
    return factorial(n) // (factorial(n - k) * factorial(k))

choose(9, 6)

84

In [280]:
N = len(U6)

# `P` computes a ratio and `choose` computes a count,
# so we can multiply the probability back by the denominator `cases(space)`, i.e. len(U6) 
# so that both are counts.
print(N * P(select('R', 6), U6) == choose(9, 6))
# which is just:
print(len(favorable(select('R', 6), U6)) == choose(9, 6))

True
True


In [281]:
# 3 are blue, and 1 is red, and 2 are white

N * P(select('B', 3) & select('W', 2) & select('R', 1), U6) == choose(6, 3) * choose(9, 1) * choose(8, 2) 

True

In [282]:
# 4 balls (only) are white
N * P(select('W', 4), U6) == choose(8, 4) * choose(6 + 9, 2)  # (6 + 9 non-white balls)

True

In [283]:
from collections import Counter
        
class Dist(Counter): 
    "A Distribution of {outcome: frequency} pairs."

# Changes to previous functions to allow use of distributions (via `Dist`):
#   - Sample spaces and events can both be specified as either a `set` or a `Dist`.
#   - The sample space can be a non-probability distribution like `Dist(H=50, T=50)`; the results
#   will be the same as if the sample space had been a true probability distribution like `Dist(H=1/2, T=1/2)`.
#   - `cases` now sums the frequencies in a distribution (it previously counted the length).
#   - `favorable` now returns a `Dist` of favorable outcomes and their frequencies (not a `set`).
#   - Redefines `Fraction` to use `"/"`, not `fractions.Fraction`, because frequencies might be floats.
#   - `P` is unchanged.

def cases(outcomes): 
    "The total frequency of all the outcomes."
    return sum(Dist(outcomes).values())

def favorable(event, space):
    "A distribution of outcomes from the sample space that are in the event."
    space = Dist(space)
    return Dist({x: space[x] 
                for x in space if x in event})

def Fraction(n, d): return n / d
    
# multiple ways to define a distribution
assert Dist(H=5, T=4) == Dist({'H': 5}, T=4) == Dist('TTTT', H=5) == Dist('THHHTTHHT')

# Example: probability of rolling an even number with a crooked die that is loaded to prefer 6:
Crooked = Dist({1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.1, 6: 0.5})
P(even, Crooked) # now using new versions of cases, favorable and Fraction


0.7

In [284]:
# vs getting an odd
P(odd, Crooked) # now using new versions of cases, favorable and Fraction


0.30000000000000004

### Allowing predicates to be used as events

To calculate the probability of an even die roll, we've been using

even = {2, 4, 6}

Which makes us alter the events if we were to change the die face number.

It is much more practical to allow events to be computed in this case and
redefine `favorable` once again:

In [285]:
def even(n): return n % 2 == 0

def favorable(event, space):
    "A distribution of outcomes from the sample space that are in the event."
    if callable(event):
        event = {x for x in space if event(x)}
    space = Dist(space)
    return Dist({x: space[x] 
                 for x in space if x in event})
    
favorable(even, D)

Dist({2: 1, 4: 1, 6: 1})

In [286]:
P(even, D)

0.5

The sample space (die) can also we computed dynamically:

In [287]:
def die(n): return set(range(1, n + 1))

favorable(even, die(12))

Dist({2: 1, 4: 1, 6: 1, 8: 1, 10: 1, 12: 1})

In [288]:
P(even, die(12))

0.5

In [289]:
P(even, die(2000))

0.5

In [290]:
P(even, die(2001))

0.49975012493753124

In [291]:
#
# Calculate the probability that the sum of rolling `d` 6-sided dice is prime:
#

def sum_dice(d): return Dist(sum(dice) for dice in itertools.product(D, repeat=d))

def is_prime(n): return (n > 1 and not any(n % i == 0 for i in range(2, n)))

for d in range(1, 9):
    p = P(is_prime, sum_dice(d))
    print(f"P(is_prime, sum_dice({d})) = {round(p, 3)}")

P(is_prime, sum_dice(1)) = 0.5
P(is_prime, sum_dice(2)) = 0.417
P(is_prime, sum_dice(3)) = 0.338
P(is_prime, sum_dice(4)) = 0.333
P(is_prime, sum_dice(5)) = 0.317
P(is_prime, sum_dice(6)) = 0.272
P(is_prime, sum_dice(7)) = 0.242
P(is_prime, sum_dice(8)) = 0.236


### Coin toss game (Fermat & Pascal)

Consider a gambling game consisting of tossing a coin repeatedly. 
Player H wins the game as soon as a total of 10 heads come up, and T wins if a
total of 10 tails come up before H wins. 
If the game is interrupted when H has 8 heads and T has 7 tails, 
how should the pot of money (which happens to be 100 Francs) be split?

In [292]:
def win_unfinished_game(h, t):
    "The probability that H will win the unfinished game, given the number of points `h` and `t` needed by H and T to win."
    return P(at_least(h, 'h'), finishes(h, t))

def at_least(n, item):
    "The event of getting at least n instances of item in an outcome."
    return lambda outcome: outcome.count(item) >= n
    
def finishes(h, t):
    "All finishes of a game where player H needs h points to win and T needs t."
    tosses = ['ht'] * (h + t - 1)
    return set(itertools.product(*tosses))

In [293]:
endings = finishes(2, 3) # current state, where H needs 2 coin tosses to win and T needs 3
# 16 equiprobable endings
endings

{('h', 'h', 'h', 'h'),
 ('h', 'h', 'h', 't'),
 ('h', 'h', 't', 'h'),
 ('h', 'h', 't', 't'),
 ('h', 't', 'h', 'h'),
 ('h', 't', 'h', 't'),
 ('h', 't', 't', 'h'),
 ('h', 't', 't', 't'),
 ('t', 'h', 'h', 'h'),
 ('t', 'h', 'h', 't'),
 ('t', 'h', 't', 'h'),
 ('t', 'h', 't', 't'),
 ('t', 't', 'h', 'h'),
 ('t', 't', 'h', 't'),
 ('t', 't', 't', 'h'),
 ('t', 't', 't', 't')}

In [294]:
# of which 11 are favorable to player H who needs 2 tosses
favorable(at_least(2, 'h'), endings)

Dist({('t', 'h', 'h', 't'): 1,
      ('t', 'h', 't', 'h'): 1,
      ('h', 'h', 't', 'h'): 1,
      ('h', 't', 'h', 'h'): 1,
      ('h', 'h', 't', 't'): 1,
      ('h', 'h', 'h', 'h'): 1,
      ('h', 't', 'h', 't'): 1,
      ('t', 't', 'h', 'h'): 1,
      ('h', 'h', 'h', 't'): 1,
      ('h', 't', 't', 'h'): 1,
      ('t', 'h', 'h', 'h'): 1})

In [295]:
# and of course only 16 - 11 are favorable to player T
favorable(at_least(3, 't'), endings)

Dist({('h', 't', 't', 't'): 1,
      ('t', 't', 't', 't'): 1,
      ('t', 'h', 't', 't'): 1,
      ('t', 't', 't', 'h'): 1,
      ('t', 't', 'h', 't'): 1})

In [296]:
# If we need to justly split 100 currency artefacts between the two players, H will get:
100 * win_unfinished_game(2, 3)

68.75

In [297]:
for t_tosses_to_win in reversed(range(10)):
    print(f"H needs 2 tosses, T needs {t_tosses_to_win} tosses - H gets: {round(100 * win_unfinished_game(2, t_tosses_to_win), 2)}%")

H needs 2 tosses, T needs 9 tosses - H gets: 98.93%
H needs 2 tosses, T needs 8 tosses - H gets: 98.05%
H needs 2 tosses, T needs 7 tosses - H gets: 96.48%
H needs 2 tosses, T needs 6 tosses - H gets: 93.75%
H needs 2 tosses, T needs 5 tosses - H gets: 89.06%
H needs 2 tosses, T needs 4 tosses - H gets: 81.25%
H needs 2 tosses, T needs 3 tosses - H gets: 68.75%
H needs 2 tosses, T needs 2 tosses - H gets: 50.0%
H needs 2 tosses, T needs 1 tosses - H gets: 25.0%
H needs 2 tosses, T needs 0 tosses - H gets: 0.0%


> Which of the following three propositions has the greatest chance of success? 
> 1. Six fair dice are tossed independently and at least one “6” appears. 
> 2. Twelve fair dice are tossed independently and at least two “6”s appear. 
> 3. Eighteen fair dice are tossed independently and at least three “6”s appear.

In [302]:
die6 = Dist({6: 1/6, '?': 5/6}) # we don't care about other possibilities

# dice rollings are independent from one another. So are coin flips. 
# Would not apply to e.g. removing balls from bags, etc. where order matters
# When order matters -> conditional probability is what we're looking at.
def joint(A, B, combine='{}{}'.format):
    """The joint distribution of two independent and identical distributions (IID). 
    Result is all entries of the form {'ab': frequency(a) * frequency(b)}"""
    return Dist({combine(a, b): A[a] * B[b]
                 for a in A for b in B})
    
def dice_prob(n, die):
    "Joint probability distribution from rolling `n` dice."
    if n == 1:
        return die
    else:
        return joint(die, dice_prob(n - 1, die))
    
dice_prob(4, die6)

Dist({'6666': 0.0007716049382716049,
      '666?': 0.0038580246913580245,
      '66?6': 0.0038580246913580245,
      '66??': 0.019290123456790126,
      '6?66': 0.0038580246913580245,
      '6?6?': 0.019290123456790126,
      '6??6': 0.019290123456790126,
      '6???': 0.09645061728395063,
      '?666': 0.0038580246913580245,
      '?66?': 0.019290123456790122,
      '?6?6': 0.019290123456790122,
      '?6??': 0.09645061728395063,
      '??66': 0.019290123456790122,
      '??6?': 0.09645061728395063,
      '???6': 0.09645061728395063,
      '????': 0.48225308641975323})

In [306]:
# going back to the original question:
print(P(at_least(1, '6'), dice_prob(6, die6)), "<-- we're better off rolling six dice and hoping for 1 six") 
print(P(at_least(2, '6'), dice_prob(12, die6)))
print(P(at_least(3, '6'), dice_prob(18, die6)))

0.665102023319616 <-- we're better off rolling six dice
0.61866737373231
0.5973456859477678



## References

- Peter Norvig's [ipynb](https://github.com/norvig/pytudes/blob/main/ipynb)
-